# Train with real data from Fluvius

## Read raw data

In [ ]:
import os
from pathlib import Path
import pandas as pd
directory = Path.cwd()
time_series_Consumption=pd.read_pickle(os.path.join(directory,'all_time_series_Consumption.pkl'))
labels_df=pd.read_pickle(os.path.join(directory,'labels_df.pkl'))
time_series_Consumption.set_index('Datetime', inplace=True)
time_series_Consumption=time_series_Consumption.resample('h').sum()

#features_to_plot = [
#    ('mean', "Mean"),
#    ('std', "Standard Deviation"),
#    ('median', "Median"),
#    ('skew', "Skewness"),
#    ('sum', "Sum"),
#    ('min', "Minimum"),
#    ('max', "Maximum")]"

## Extract features

### Fuction for extarcting features
Featue engineering has been done in a seperate excerceis with a collection of scripts in the file named development.ipynb

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Define a function to extract weekly and monthly features with prefixed row names and a single 'feature' column
def extract_time_features(df, freq, prefix):
    # Calculate statistics for each period individually
    mean = df.resample(freq).mean()
    mean.index = [f"{prefix}_mean_{i}" for i in range(len(mean))]
    
    std = df.resample(freq).std()
    std.index = [f"{prefix}_std_{i}" for i in range(len(std))]
    
    skew = df.resample(freq).apply(pd.Series.skew)
    skew.index = [f"{prefix}_skew_{i}" for i in range(len(skew))]
    
    sum_ = df.resample(freq).min()
    sum_.index = [f"{prefix}_sum_{i}" for i in range(len(sum_))]
    
    max_ = df.resample(freq).max()
    max_.index = [f"{prefix}_max_{i}" for i in range(len(max_))]
    
    # Concatenate all features
    features = pd.concat([mean, std, sum_, skew, max_], axis=0) #min_,
    
    # Reset the index to have 'feature' as a column and IDs as columns
    features = features.reset_index()
    features = features.rename(columns={'index': 'feature'})
    
    return features


### Train, test and validation sets

In [ ]:

# Extract weekly and monthly features for each ID
weekly_features = extract_time_features(time_series_Consumption, 'W', 'W')
monthly_features = extract_time_features(time_series_Consumption, 'ME', 'M')
Annual_features = extract_time_features(time_series_Consumption, 'YE', 'A')
# Combine weekly and monthly features into a single DataFrame
combined_features = pd.concat([weekly_features, monthly_features,Annual_features], axis=0).reset_index(drop=True)

new_column_names = combined_features['feature'].values
combined_features_temp= combined_features.drop(columns=['feature']).transpose()
combined_features_temp.columns = new_column_names
combined_features_temp=combined_features_temp.reset_index().rename(columns={'index': 'ID'})
combined_features_temp.ID=combined_features_temp.ID.astype(int)
# Step 1: Merge features and labels
data = pd.merge(combined_features_temp, labels_df[['ID', 'Category']], on='ID')

# Step 2: Separate features (X) and target (y)
X = data.drop(columns=['Category','ID'])
y = data['Category']

# seperate part of the data for validation 
X_org, X_valid,y_org, y_valid = train_test_split(X, y, test_size=0.2, random_state=80)

## Model trainin

### Hyper parameter optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import optuna
import optuna.visualization as vis

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

# ========== [1] Encode labels ==========
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_org)

# ========== [2] Define Optuna objective functions ==========
#A colmplete list of hyper parameters
#params = {
        #'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
       # 'max_depth': trial.suggest_int('max_depth', 3, 10),
#        'n_estimators': trial.suggest_int('n_estimators', 5, 350),
        #'subsample': trial.suggest_float('subsample', 0.6, 1.0),
       # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
       # 'gamma': trial.suggest_float('gamma', 0, 1),
        #'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
#        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
       #'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        #'use_label_encoder': False,
        #'eval_metric': 'mlogloss'
#    }

def objective_xgb(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'use_label_encoder': False,
        'eval_metric': 'mlogloss'
    }

    trial.set_user_attr("model_type", "xgb")
    model = XGBClassifier(**params)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_org, y_encoded, cv=cv, scoring='accuracy').mean()
    return score


def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2'])
    }

    trial.set_user_attr("model_type", "rf")
    model = RandomForestClassifier(**params, random_state=42)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(model, X_org, y_encoded, cv=cv, scoring='accuracy').mean()
    return score

# ========== [3] Run the Optuna study ==========
# Choose which model to tune use xgb or rf (random forest)
use_model = "xgb" # "rf" # 

study = optuna.create_study(direction='maximize')

if use_model == "xgb":
    study.optimize(objective_xgb, n_trials=50)
else:
    study.optimize(objective_rf, n_trials=50)

# ========== [4] Train the final model ==========
print("Best Accuracy:", study.best_value)
print("Best Parameters:", study.best_params)

model_type = study.best_trial.user_attrs.get("model_type")

if model_type == "xgb":
    final_model = XGBClassifier(**study.best_params)
elif model_type == "rf":
    final_model = RandomForestClassifier(**study.best_params, random_state=42)
else:
    raise ValueError("Unknown model type in study.")

final_model.fit(X_org, y_encoded)

# ========== [5] Save model and label encoder ==========
joblib.dump(final_model, f"best_{model_type}_model.joblib")
joblib.dump(label_encoder, 'label_encoder.joblib')

# ========== [6] Visualize tuning results ==========
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()


In [ ]:
print("Best Accuracy:", study.best_value)
print("Best Parameters:", study.best_params)

# [5] Train final model using best hyperparameters
best_params = study.best_params
best_params['use_label_encoder'] = False
best_params['eval_metric'] = 'mlogloss'

final_model = XGBClassifier(**study.best_params, random_state=80)
final_model.fit(X_org, y_encoded)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Make predictions using the best model
y_pred = final_model.predict(X_valid)
y_valid_encoded=label_encoder.fit_transform(y_valid)

# Step 2: Calculate overall accuracy
overall_accuracy = accuracy_score(y_valid_encoded, y_pred)
print(f"Overall Accuracy: {overall_accuracy:.2f}")

# Step 3: Detailed classification report for per-label accuracy (precision, recall, F1-score for each label)
classification_report_dict = classification_report(y_valid_encoded, y_pred, output_dict=True)
classification_report_df = pd.DataFrame(classification_report_dict).transpose()

# Display per-label accuracy (recall for each label)
print("\nClassification Report:\n")
print(classification_report_df)

# Plot per-label accuracy (recall)
plt.figure(figsize=(10, 6))
sns.barplot(x=classification_report_df.index[:-3], y=classification_report_df['recall'][:-3], palette="viridis")
plt.title("Per-Label Accuracy (Recall)")
plt.xlabel("Labels")
plt.ylabel("Recall")
plt.xticks(rotation=45, ha="right")
plt.show()

# Step 4: Confusion Matrix
cm = confusion_matrix(y_valid_encoded, y_pred, labels=final_model.classes_)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Step 5: Feature Importance (Top 10)
# Get feature importances from the best model
feature_importances = final_model.feature_importances_
features = X_org.columns  # Assuming `X_train` has the original features
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Sort by importance and select the top 10 features
top_10_importances = importance_df.sort_values(by='Importance', ascending=False).head(10)

# Plot the top 10 feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=top_10_importances, palette="viridis")
plt.title("Top 10 Feature Importances")
plt.show()
